# Ecological Metadata Language (EML)

Some archives include additional metadata stored in the form of an XML file that uses [Ecological Metadata Language (EML)](https://eml.ecoinformatics.org/).  This path to this file can be retrieved from the `ArchiveReader.MetaData`.  This notebook describes how information can be retrieved from the dataset metadata and presented.

In [1]:
#r "nuget:DwC-A_dotnet,0.7.0"
#r "nuget:DwC-A_dotnet.Interactive,0.1.10-Pre"

Installed Packages DwC-A_dotnet, 0.7.0 DwC-A_dotnet.Interactive, 0.1.10-pre

Loading extensions from `DwC-A_dotnet.Interactive.dll`

In [2]:
using DwC_A;
using System.IO;

var archive = new ArchiveReader("./data/dwca-rooftop-v1.4.zip");
var emlPath = Path.Combine(archive.OutputPath, archive.MetaData.Metadata);
emlPath

C:\Users\pj865\AppData\Local\Temp\dwca\d57788d7-19f5-43a7-857e-5145d5d0c465\eml.xml

## Using XPath Queries

Using an XPathNavigator object we can query into any part of the document and retrieve single elements or sections.

For some tips on finding the information you need see the document [Best Practices for Dataset Metadata in Ecological Metadata Language (EML)](https://environmentaldatainitiative.org/five-phases-of-data-publishing/phase-3/metadata-best-practices/) available for download from [https://environmentaldatainitiative.org](https://environmentaldatainitiative.org).

In [3]:
using System.Xml;
using System.Xml.XPath;

var docNav = new XPathDocument(emlPath);
var nav = docNav.CreateNavigator();
XmlNamespaceManager nsmgr = new XmlNamespaceManager(nav.NameTable);
nsmgr.AddNamespace("eml", "eml://ecoinformatics.org/eml-2.1.1");
var root = nav.SelectSingleNode("eml:eml", nsmgr);

var title = root.SelectSingleNode("dataset/title", nsmgr).ToString();
title

Insects from light trap (1992–2009), rooftop Zoological Museum, Copenhagen

## Formatter Shortcut

The following Formatters can be used as a shortcut to display the results of simple queries.  Try some of the following in the cell following the next.

* dataset/title
* dataset/abstract/para
* dataset/keywordSet/keyword
* additionalMetadata/metadata/gbif/citation

In [4]:
using Microsoft.DotNet.Interactive.Formatting;
using System.Xml.XPath;

using static  Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

Formatter.Register<XPathNavigator>((nav, writer) => {
    var output = p(nav.ToString());
    writer.Write(output);
}, "text/html");

Formatter.Register<XPathNodeIterator>((iter, writer) => {
    var outputList = new List<dynamic>();
    foreach(var node in iter)
    {
        outputList.Add(p(node.ToString()));
    }
    writer.Write(div(outputList));
}, "text/html");

In [5]:
root.Select("dataset/title", nsmgr)

Insects from light trap (1992–2009), rooftop Zoological Museum, Copenhagen

## Using XSL Stylesheets

We can also perform XSL Transforms on the data to produce an HTML document using the XslCompiledTransform class as follows.  Here we are using a the stylesheet eml.xslt to retrieve the document title, abstract and citation information.

In [6]:
using System.Xml.Xsl;
using System.Xml;
using Microsoft.AspNetCore.Html;

using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

using(var stringWriter = new StringWriter())
using(var writer = new XmlTextWriter(stringWriter))
{
    XslCompiledTransform xslt = new XslCompiledTransform();
    xslt.Load("./data/eml.xslt");
    xslt.Transform(emlPath, writer);
    display(new HtmlString(stringWriter.ToString()));
}

Meta Data Insects from light trap (1992–2009), rooftop Zoological Museum, Copenhagen Abstract Collecting of Lepidoptera and Coleoptera was carried out with an automatically working modified Robinson light trap, which was installed 17.5 m above ground at the roof of the Zoological Museum in Copenhagen, Denmark. The trap worked around 4,500 nights from April to November during the years 1992 to 2009, and was emptied on an approximately weekly basis. An analysis of this dataset was recently published in the Journal of Animal Ecology: https://dx.doi.org/10.1111/1365-2656.12452 The original data was deposited in Dryad and has now been republished in the standard Darwin Core Archive format using the IPT. Also note that the Lepidoptera data underwent a number of minor name corrections. Since new collaborations are extremely valuable to make the most of the data, researchers are encouraged to contact the dataset creator to collaborate on joint analyses and meta-analyses. Citation Karlsholt O, Pedersen J, Hansen (deceased) M (2015): Insects from light trap (1992–2009), rooftop Zoological Museum, Copenhagen. v1.4. Natural History Museum of Denmark. Dataset/Samplingevent. http://doi.org/10.15468/xabmiz